In [13]:
import pandas as pd
df = pd.read_csv('C:\\Users\\Work\\Downloads\\DATASETS\\FoCS_project\\data.csv')
#print(df.head())

for col, dtype in df.dtypes.items():
    print(f"{col}: {dtype}")

C:\Users\Work\AppData\Local\Temp\ipykernel_15572\1223733496.py:2: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('C:\\Users\\Work\\Downloads\\DATASETS\\FoCS_project\\data.csv')


VendorID: float64
tpep_pickup_datetime: object
tpep_dropoff_datetime: object
passenger_count: float64
trip_distance: float64
RatecodeID: float64
store_and_fwd_flag: object
PULocationID: int64
DOLocationID: int64
payment_type: float64
fare_amount: float64
extra: float64
mta_tax: float64
tip_amount: float64
tolls_amount: float64
improvement_surcharge: float64
total_amount: float64
congestion_surcharge: float64


Extract all trips with trip_distance larger than 50

In [ ]:
long_trips = df[df['trip_distance'] > 50]
print(long_trips.head())

Extract all trips where payment_type is missing

In [ ]:
missing_payment = df[df['payment_type'].isnull()]
print(missing_payment.head())

For each (PULocationID, DOLocationID) pair, determine the number of trips

In [ ]:
trip_counts = df.groupby(['PULocationID', 'DOLocationID']).size().reset_index(name='trip_count')
print(trip_counts)

Save all rows with missing VendorID, passenger_count, store_and_fwd_flag, payment_type in a new dataframe called bad, and remove those rows from the original dataframe.

In [11]:
# Define columns to check for missing values
columns_to_check = ['VendorID', 'passenger_count', 'store_and_fwd_flag', 'payment_type']

# Create 'bad' DataFrame with missing values in any of those columns
bad_df = df[df[columns_to_check].isna().any(axis=1)] #Look across columns in each row. If any column is missing, return True for that row.

# Remove those rows from the original DataFrame
df = df.drop(bad_df.index)

Add a duration column storing how long each trip has taken (use tpep_pickup_datetime, tpep_dropoff_datetime)

In [12]:
# Convert datetime columns
df['tpep_pickup_datetime'] = pd.to_datetime(df['tpep_pickup_datetime'])
df['tpep_dropoff_datetime'] = pd.to_datetime(df['tpep_dropoff_datetime'])

# Add duration column in minutes
df['duration'] = (df['tpep_dropoff_datetime'] - df['tpep_pickup_datetime']).dt.total_seconds() / 60

For each pickup location, determine how many trips have started there.

In [14]:
# Count number of trips per pickup location
pickup_counts = df.groupby('PULocationID').size().reset_index(name='trip_count')

# Display result
print(pickup_counts)

     PULocationID  trip_count
0               1         755
1               2           3
2               3         206
3               4       10147
4               5          39
..            ...         ...
256           261       34583
257           262       85805
258           263      124733
259           264       43958
260           265        3160

[261 rows x 2 columns]


Cluster the pickup time of the day into 30-minute intervals (e.g. from 02:00 to 02:30)

In [ ]:
# Make sure the column is datetime
df['tpep_pickup_datetime'] = pd.to_datetime(df['tpep_pickup_datetime'])

# Create a new column with 30-minute intervals
df['pickup_half_hour'] = df['tpep_pickup_datetime'].dt.floor('30min')

pickup_distribution = df.groupby('pickup_half_hour').size().reset_index(name='trip_count')
print(pickup_distribution.head())